## Abstract

Many statistical populations contain distinct subpopulations, where each one may posses its own density. This can result in a multimodal distribution which can not be modelled by a single density. Modelling mixtures of distributions has key applications in document topic analysis and financial crisis modelling.

Gaussian Mixture Models, used to model subpopulations, are convex combinations of Gaussian densities. Parameters for this model can be estimated using the Expectation Maximization (EM) algorithm - a probablistic approach.

Recent discoveries in optimization research have yielded genetic algorithms inspired by processes found in nature. One particular example is Particle Swarm Optimization, which we implement to perform maximum likelihood estimation. We compare this algorithm to the classical EM by applying them to the Old Faithful dataset and to simulated mixture densities.

Particularly, we aim to compare this non-statistical approach (PSO) to the probablisitic approach (EM) to estimate densities for the univariate case where there are two distinct groups.


## Introduction

In practise, datasets are generated from process which can be represented as a combination of Gaussian distributions. The parameters of the distribution that generates these samples can offer insights into properties of the population, such as the subpopulation means and the weights of each mixture component. Concretely, this mixture distribution represents a decomposition of a single density into a convex combination of many. 

We formally define a Gaussian Mixture Model pdf $f$ with two components as

$$
f(x\space|\space\mu_{1},\sigma_{1}^{2},\mu_{2},\sigma_{2}^{2}) = \pi\phi(x\space|\space\mu_{1},\sigma_{1}^{2}) + (1-\pi)\phi(x\space|\space\mu_{2},\sigma_{2}^{2})
$$

where $\phi_{1},\phi_{2}$ are the Gaussian pdf and the weight $\pi \in [0,1]$.

This formulation describes two Gaussian components, each with a weight and their respective parameters $\mu$ and $\sigma_{2}$ being their mean and variance respectively.

We often do not know these parameters of the population given a sample dataset. At first glance, an approach would be to determine the log-likelihood function, yielding


$$
l(\mu_{1},\sigma_{1}^{2},\mu_{2},\sigma_{2}^{2}|x) = 
\sum_{i=1}^{n}[\ln(\pi\phi(x_{i}|\mu_{1},\sigma_{1}^{2}) + (1-\pi)\phi(x_{i}|\mu_{2},\sigma_{2}^{2}))]
$$

Maximum likelihood estimation by setting the score function to zero would require solving a multivariate optimization problem. This problem is challenging due to multiple variables, the complexity of the density $\phi$, and the summation of these. Thus numerical algorithms are required to find an approximate solution.


The most widely used approach is the Expectation Maximization (EM) algorithm (CITE TEXTBOOK HERE). 
It is a deterministic optimization technique derived from minorize-maximization (MM) methods. We say the log-likelihood function is maximized by by iteratively maximizing simpler functions. 

(?? ADD ALGO LATER???)



PSO = inspiration etc

We propose to compare this to the Particle Swarm Optimization - a stochastic optimization algorithm first described by Kennedy (CITE HIM) in 1995. It draws inspirations from behaviour seen in crowds of lifeforms such as school of fish or flocks of birds. The main idea is to begin with an initial 'swarm' of particles with values randomly chosen from the search space. Each particle represents a candidate solution to the optimization problem - each with its own 'inertia' which is a step size and direction. At each iteration the particles move in a direction determined by linear combination of its inertia and the global best solution found so far. After multiple iterations of this, the algorithm will terminate when the particles have converged to an optimum.

The comparison of the PSO and the EM will be the focus of the remainder of the paper. In the following section, we present the respective implementations of the methods. Then, we bring forth two ways of performing the compaarison, namely by applying our implementations to the well known Old Faithful dataset. To further quantify our comparison, we also perform Monte Carlo simulation. We end our report with the discussion of our results and potential for future improvements and research 

It was first described by Dempster et al. in 1977. 

Hype about non-statistical algos 




example figure here? :O

- comparisons between these two methods via repeated applications of thhe algooirthm to simulated datasets
- mention PSO typically used for opt problems and we wish to see how it performs in statistical density estimation 
- non-statistical methods for machine learning and estimation are gaining traction and we wish to test that shit out (cite XXX)

- in the remainder of this paper we implement 1995's PSO and X articles EM (textbook) and 


## Methodology

We formally define a Gaussian Mixture Model as

$$
XXX
$$



- compare PSO and EM algos and discuss implemetnation
	- math behind PSO and how the vector is updated
	- EM formulas

- show math for likelihood function
- implemntation details
- will we quantify the comparision using
- assumptions/restrictions on our simulated params: only two mixtures
- monte carlo thing (simulated data)
	- generate multiple datasets from known mixture distributions
	- we can measure accuracy/error by comparing the resulting params to the the true ones

- how we compare
- discuss error measurements

- application of analysis to a real dataset (faithful data)
	- compare both 
	- plot the estimated densities vs the original dataset
	- simulate data from estimated mixtures and compare to original dataset - compare densities


## Results - Single Dataset

## Results - Multiple Simulated Datasets